# Azure AI Search: Relevance

## Setup API clients

In [5]:
import os

import azure.identity
import dotenv
import openai
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizedQuery

dotenv.load_dotenv()

azure_credential = azure.identity.AzureDeveloperCliCredential(tenant_id=os.environ["AZURE_TENANT_ID"])

# Initialize Azure OpenAI client
AZURE_OPENAI_SERVICE = os.getenv("AZURE_OPENAI_SERVICE")
AZURE_OPENAI_EMBEDDING_DEPLOYMENT = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT")

token_provider = azure.identity.get_bearer_token_provider(azure_credential, "https://cognitiveservices.azure.com/.default")
openai_client = openai.AzureOpenAI(
    api_version="2023-07-01-preview",
    azure_endpoint=f"https://{AZURE_OPENAI_SERVICE}.openai.azure.com",
    azure_ad_token_provider=token_provider)

def get_embedding(text):
    get_embeddings_response = openai_client.embeddings.create(model=AZURE_OPENAI_EMBEDDING_DEPLOYMENT, input=text)
    return get_embeddings_response.data[0].embedding

# Initialize Azure search client
AZURE_SEARCH_SERVICE = os.getenv("AZURE_SEARCH_SERVICE")
AZURE_SEARCH_ENDPOINT = f"https://{AZURE_SEARCH_SERVICE}.search.windows.net"

AZURE_SEARCH_INDEX = os.environ["AZURE_SEARCH_INDEX"]
search_client = SearchClient(AZURE_SEARCH_ENDPOINT, AZURE_SEARCH_INDEX, credential=azure_credential)


## Keyword search

In [17]:
search_query = "25 foot hose"
r = search_client.search(search_text=search_query, top=5)
for doc in r:
    content = doc["content"].replace("\n", " ")[:150]
    print(f"Score: {doc['@search.score']:.5f}\tContent:{content}")

Score: 15.90960	Content:Soaker Hose 25-foot SKU: GOHOS002 Price: $25.00 Category: GARDEN & OUTDOOR > HOSES   <figure><figcaption>1.1 <br>The image shows a coiled black hose o
Score: 10.33728	Content:Garden Hose 50-foot  SKU: GOHOS001 Price: $32.00 Category: GARDEN & OUTDOOR > HOSES   <figure><figcaption>1.1 <br>The image shows a coiled metal hose 
Score: 7.63983	Content:   Heavy-duty garden hose with brass fittings and kink-resistant construction.
Score: 7.04795	Content:   Porous soaker hose for efficient water delivery directly to plant roots.
Score: 4.35788	Content:# Liquid-Tight Conduit  SKU: ELCO000010 Price: $20.00 Category: ELECTRICAL > CONDUIT   <figure><figcaption>1.1 <br>The image shows a coiled, metallic,


## Vector search


In [18]:
search_query = "garden watering supplies"
search_vector = get_embedding(search_query)
r = search_client.search(None, top=3, vector_queries=[
    VectorizedQuery(vector=search_vector, k_nearest_neighbors=50, fields="embedding3")])
for doc in r:
    content = doc["content"].replace("\n", " ")[:150]
    print(f"Score: {doc['@search.score']:.5f}\tContent:{content}")

Score: 0.70706	Content:Misting Sprinkler Kit SKU: GOSPR004 Price: $28.00 Category: GARDEN & OUTDOOR > SPRINKLERS   <figure><figcaption>1.1 <br>The image shows an arrangement
Score: 0.66958	Content:In-Ground Sprinkler Head SKU: GOSPR005 Price: $12.00 Category: GARDEN & OUTDOOR > SPRINKLERS   <figure><figcaption>1.1 <br>The image shows a black cyl
Score: 0.66597	Content:Soaker Hose 25-foot SKU: GOHOS002 Price: $25.00 Category: GARDEN & OUTDOOR > HOSES   <figure><figcaption>1.1 <br>The image shows a coiled black hose o


## Hybrid search with RRF

In [19]:
search_vector = get_embedding(search_query)
r = search_client.search(search_query, top=15, vector_queries=[
    VectorizedQuery(vector=search_vector, k_nearest_neighbors=10, fields="embedding3")])
for doc in r:
    content = doc["content"].replace("\n", " ")[:150]
    print(f"Score: {doc['@search.score']:.5f}\tContent:{content}")

Score: 0.03150	Content:   Complete misting system for cooling and plant watering with multiple nozzles.
Score: 0.03137	Content:Misting Sprinkler Kit SKU: GOSPR004 Price: $28.00 Category: GARDEN & OUTDOOR > SPRINKLERS   <figure><figcaption>1.1 <br>The image shows an arrangement
Score: 0.03089	Content:Landscape Spotlight Kit  SKU: GOOTD004 Price: $52.00 Category: GARDEN & OUTDOOR > OUTDOOR LIGHTING   <figure><figcaption>1.1 <br>The image shows a nea
Score: 0.03078	Content:   Heavy-duty garden hose with brass fittings and kink-resistant construction.
Score: 0.03062	Content:Soaker Hose 25-foot SKU: GOHOS002 Price: $25.00 Category: GARDEN & OUTDOOR > HOSES   <figure><figcaption>1.1 <br>The image shows a coiled black hose o
Score: 0.03054	Content:Garden Hose 50-foot  SKU: GOHOS001 Price: $32.00 Category: GARDEN & OUTDOOR > HOSES   <figure><figcaption>1.1 <br>The image shows a coiled metal hose 
Score: 0.03048	Content:In-Ground Sprinkler Head SKU: GOSPR005 Price: $12.00 Category: GARDEN & OUT

## Hybrid with re-ranker

In [15]:
search_query = "garden watering supplies"
search_vector = get_embedding(search_query)
r = search_client.search(
        search_query,
        top=5, 
        vector_queries=[
                VectorizedQuery(vector=search_vector, k_nearest_neighbors=50, fields="embedding3")],
        query_type="semantic",
        semantic_configuration_name="default")

for doc in r:
    content = doc["content"].replace("\n", " ")[:150]
    print(f"Score: {doc['@search.score']:.5f}\tReranker: {doc['@search.reranker_score']:.5f}\tContent: {content}")

Score: 0.03125	Reranker: 2.49573	Content:    Heavy-duty garden hose with brass fittings and kink-resistant construction.
Score: 0.03102	Reranker: 2.46081	Content: Garden Hose 50-foot  SKU: GOHOS001 Price: $32.00 Category: GARDEN & OUTDOOR > HOSES   <figure><figcaption>1.1 <br>The image shows a coiled metal hose 
Score: 0.03159	Reranker: 2.43777	Content: Misting Sprinkler Kit SKU: GOSPR004 Price: $28.00 Category: GARDEN & OUTDOOR > SPRINKLERS   <figure><figcaption>1.1 <br>The image shows an arrangement
Score: 0.03200	Reranker: 2.28960	Content:    Complete misting system for cooling and plant watering with multiple nozzles.
Score: 0.01538	Reranker: 2.22509	Content:    Pop-up sprinkler head for permanent irrigation systems with adjustable arc.
